# Harmony Service Chaining: PI 20.4 Demo

In PI 20.4, Harmony add service chaining capabilities to support requests that require functionality beyond that of a single service.
This notebook provides a basic workflow to demonstrate service chaining. For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb).  Useful helpers for making the calls found in this notebook can be found under the [docs/notebook-helpers](./notebook-helpers) folder.

## Prerequisites

#### You must run this notebook within an EC2 instance running in us-west-2:
1. Follow tutorials 01 through 03 of the [NASA Earthdata Cloud Primer](https://earthdata.nasa.gov/learn/user-resources/webinars-and-tutorials/cloud-primer) to set up an EC2 instance within us-west-2. Ensure you are also following step 3 in the ["Jupyter Notebooks on AWS EC2 in 12 (mostly easy) steps"](https://medium.com/@alexjsanchez/python-3-notebooks-on-aws-ec2-in-15-mostly-easy-steps-2ec5e662c6c6) article to set the correct security group settings needed to connect your local port to your EC2’s notebook port thru SSH.

2. Follow the remaining instructions in the Medium article above, which includes installation of Anaconda3 (including Jupyter Lab) in your ec2 instance. Before moving over to Jupyter Lab, perform steps 3 - 4 to set up Earthdata Login and Harmony access:

3. Setup your `~/.netrc` for Earthdata Login in your ec2 instance:

`machine uat.urs.earthdata.nasa.gov login <user> password <password>`

4. Run the following in your ec2 instance terminal window to generate short-term Harmony access keys:

`curl -Ln -bj https://harmony.uat.earthdata.nasa.gov/cloud-access.sh`

5. Set your environment variables based on the keys provided in step 4:

`export AWS_ACCESS_KEY_ID='...
export AWS_SECRET_ACCESS_KEY='...'
export AWS_SESSION_TOKEN='...'
export AWS_DEFAULT_REGION='us-west-2'`

6. Once the notebook is running in Jupyter Lab, run the following cell to install Python dependencies, import necessary modules, and set notebook defaults:

In [6]:
%load_ext autoreload
%autoreload
%matplotlib inline

import sys
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r notebook_helpers/requirements.txt

# Import libraries used throughout the notebook
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, show_shape, print_async_status, check_bbox_subset, check_stac

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Chained Services - PODAAC L1 Subsetter -> Harmony NetCDF to Zarr

This request asks for variable subsetting of L1 data with output in the Zarr format. This requires chaining two services together, the PODAAC L1 Subsetter and the Harmony NetCDF to Zarr service.

In [7]:
podaac_collection = 'C1234208438-POCLOUD'
# coverages_root = 'https://harmony.uat.earthdata.nasa.gov/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'
coverages_root = 'http://localhost:3000/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'

### Variable and spatial subsetting with reformtatting output to Zarr and spatial constraints
Each parent NetCDF is approx. 60 MB and the subsetted pngs and geotiffs are well under 1 MB each.

In [9]:
response = get(
    coverages_root.format(
        collection=podaac_collection, 
        variable='mean%5Fsea%5fsurface'),
    params={
        'format': 'application/x-zarr',
        'subset': [
            'lon(-160:-160)', 
            'lat(-80:80)'
            ],
        'maxResults': '2'})

show_async(response)

GET /C1234208438-POCLOUD/ogc-api-coverages/1.0.0/collections/mean_sea_surface/coverage/rangeset?format=application%2Fx-zarr&subset=lon%28-160%3A-160%29&subset=lat%28-80%3A80%29&maxResults=2
Async response at 18:07:43
{
  "username": "jnorton1",
  "status": "running",
  "message": "CMR query identified 7154 granules, but the request has been limited to process only the first 2 granules because you requested 2 maxResults.",
  "progress": 0,
  "createdAt": "2020-12-03T23:07:43.204Z",
  "updatedAt": "2020-12-03T23:07:43.204Z",
  "links": [
    {
      "title": "Job Status",
      "href": "http://localhost:3000/jobs/71333f77-5989-46e2-bba6-4752062b88d7",
      "rel": "self",
      "type": "application/json"
    }
  ],
  "request": "http://localhost:3000/C1234208438-POCLOUD/ogc-api-coverages/1.0.0/collections/mean_sea_surface/coverage/rangeset?format=application%2Fx-zarr&subset=lon(-160%3A-160)&subset=lat(-80%3A80)&maxResults=2",
  "jobID": "71333f77-5989-46e2-bba6-4752062b88d7"
}
Waiting fo

STACError: http://localhost:3000/stac/71333f77-5989-46e2-bba6-4752062b88d7/ does not exist locally